In [16]:
import pandas as pd
import xgboost
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [30]:
# load data
train = pd.read_csv('train.csv')
val = pd.read_csv('validation.csv')

In [4]:
train.head()

,Unnamed: 0,Unnamed: 0.1,RecordID,Age,Gender,Height,ICUType,WeightMin,WeightMax,WeightMean,...,ALPCountDis,ALTCountDis,ASTCountDis,BilirubinCountDis,FiO2MinDis,FiO2MeanDis,GCSMaxDis,PaCO2MinDis,PaCO2MaxDis,PaCO2MeanDis
0,0,0,132539.0,54.0,0.0,170.460849,4.0,81.084721,84.04358,83.110662,...,0,0,0,0,0,1,0,0,0,0
1,1,1,132540.0,76.0,1.0,175.300000,2.0,76.000000,81.60000,80.670588,...,0,0,0,0,1,0,0,1,1,1
2,2,2,132541.0,44.0,0.0,170.460849,3.0,56.700000,56.70000,56.700000,...,0,0,0,0,1,0,1,1,1,1
3,3,3,132543.0,68.0,1.0,180.300000,3.0,84.600000,84.60000,84.600000,...,0,0,0,0,0,1,0,0,0,0
4,4,4,132545.0,88.0,0.0,170.460849,3.0,81.084721,84.04358,83.110662,...,0,0,0,0,0,1,0,0,0,0


In [31]:
val.head()

,Unnamed: 0,Unnamed: 0.1,RecordID,Age,Gender,Height,ICUType,WeightMin,WeightMax,WeightMean,...,ALPCountDis,ALTCountDis,ASTCountDis,BilirubinCountDis,FiO2MinDis,FiO2MeanDis,GCSMaxDis,PaCO2MinDis,PaCO2MaxDis,PaCO2MeanDis
0,0,0,152871.0,71.0,1.0,167.600000,4.0,79.2,89.6,85.812500,...,0,0,0,0,1,0,0,1,1,1
1,1,1,152873.0,71.0,0.0,165.100000,4.0,70.0,81.7,79.641379,...,0,0,0,0,1,0,0,1,1,1
2,2,2,152875.0,65.0,0.0,169.515098,2.0,71.9,76.7,74.797778,...,0,0,0,0,1,0,0,0,0,0
3,3,3,152878.0,56.0,1.0,169.515098,4.0,86.4,86.4,86.400000,...,0,0,0,0,1,0,0,0,0,0
4,4,4,152882.0,51.0,1.0,169.515098,4.0,107.0,107.0,107.000000,...,0,0,0,0,0,1,0,1,1,1


In [56]:
X = train.iloc[:, 3:118]
Y = train.iloc[:, 118]
val_pred = val.iloc[:, 3:118]
Outcome = val.iloc[:, 118]

In [35]:
val_pred.head()

,Age,Gender,Height,ICUType,WeightMin,WeightMax,WeightMean,WeightCount,AlbuminMin,AlbuminMax,...,TempMean,TempCount,UrineMin,UrineMax,UrineMean,UrineCount,WBCMin,WBCMax,WBCMean,WBCCount
0,71.0,1.0,167.600000,4.0,79.2,89.6,85.812500,48.0,1.700000,1.900000,...,37.104167,48.0,10.0,60.0,24.225000,40.0,8.3,9.8,9.166667,3.0
1,71.0,0.0,165.100000,4.0,70.0,81.7,79.641379,29.0,2.867323,2.984308,...,37.227083,48.0,4.0,120.0,50.595745,47.0,12.6,17.8,15.366667,3.0
2,65.0,0.0,169.515098,2.0,71.9,76.7,74.797778,45.0,2.867323,2.984308,...,36.120000,10.0,8.0,1400.0,143.666667,39.0,10.5,14.7,12.600000,3.0
3,56.0,1.0,169.515098,4.0,86.4,86.4,86.400000,1.0,2.867323,2.984308,...,37.660000,15.0,15.0,260.0,68.593750,32.0,10.5,21.1,14.266667,3.0
4,51.0,1.0,169.515098,4.0,107.0,107.0,107.000000,1.0,2.867323,2.984308,...,37.520000,10.0,40.0,260.0,148.000000,30.0,6.5,8.0,7.250000,2.0


In [36]:
kfold = KFold(n_splits=10, random_state=7)
params = {'eta': 0.5, 'max_depth': 20, 'objective': 'binary:logistic',
          'eval_metric': 'logloss', 'alpha': 2}

In [50]:
preds = np.zeros([val.shape[0],])

for i, (train_id, test_id) in enumerate(kfold.split(df)):
    X_train, X_CV, y_train, y_CV = X.iloc[train_id], X.iloc[test_id], Y.values[train_id], Y.values[test_id]
    model = xgboost.train(params, xgboost.DMatrix(X_train, y_train), 10, verbose_eval=False)
    preds += model.predict(xgboost.DMatrix(val_pred))/10
    pred = model.predict(xgboost.DMatrix(X_CV))
    print('logloss :', np.sqrt(metrics.mean_squared_error(pred, y_CV)))

[20:39:42] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=15
[20:39:42] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=10
[20:39:42] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 192 extra nodes, 0 pruned nodes, max_depth=11
[20:39:42] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=12
[20:39:42] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 0 pruned nodes, max_depth=14
[20:39:42] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=10
[20:39:42] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 138 extra 

[20:39:43] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 190 extra nodes, 0 pruned nodes, max_depth=13
[20:39:43] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 176 extra nodes, 0 pruned nodes, max_depth=10
[20:39:43] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=13
[20:39:43] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=10
[20:39:43] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=11
[20:39:43] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=8
[20:39:43] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra n

In [61]:
score = 0
for prob in range(5,95,5):
    actual_pred = []
    for row in preds:
        if row >= prob/100:
            actual_pred.append(1)
        else:
            actual_pred.append(0)
    sen = metrics.recall_score(Outcome, actual_pred)
    pp = metrics.precision_score(Outcome, actual_pred)
    if min([sen, pp]) > score:
        score = min([sen, pp])
print(score)

0.48183254344391785
